## 导入模块

<!-- 一、残值系数
残值计算旧版：iPhone 采购价*0.6；非iPhone 采购价*0.4;
残值计算新版：iPhone15 最新采购价*0.4，iPhone16 最新采购价*0.6；非iPhone 采购价*0.425; -->


<!-- 二、毛利的计算逻辑
1、实际毛利计算：
订单状态为：已逾期、已买断、已完成
实际毛利 = 总已收-采购成本

订单状态为：还款中、续租中、未到首期还款日
实际毛利 = 总应收-采购成本


2、应收毛利计算：
应收毛利 = 总应收-采购成本

总已收 = 已收租金+已归还残值/已付买断尾款 + 违约金 +其他账单记账总和（滞纳金或其他扣款）
总应收 = 总租金 + 预估残值 + 违约金 -- note 20241213 增加违约金

1）租后表总已收需要查看残值表，如果有已处理完成的实际残值，用实际残值，公式如下：
总已收 = 已收租金+已归还残值/已付买断尾款（不为0）
2）租后表总如果没有已处理完成的实际残值，用预估残值，公式如下：
总已收 = 已收租金+预估残值（buyout_balance_payment） 已归还残值/已付买断尾款（为0） -->

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time , datetime

from collections import Counter
from numpy import nan as NA
from dateutil.parser import parse
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 定义数据库连接

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="huangxq",
          password="f_epz6hDmZ0Hzi0OzI0N",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 获取租后数据

In [3]:
#   `status` int DEFAULT NULL COMMENT '状态 1,待支付,2,待发货,3,待收货,4,租赁中,5,待归还,6,待商家收货/归还中,8,已完成,10,已退款,\r12,待审核,13,订单取消, 15,检测中',
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,om.order_tag, om.has_actual, tod.sku_attributes, too.money
from db_digua_business.t_postlease_receivables_monitoring   tprm
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join (SELECT t.order_id,sum(case when t.capital_type =6 and t.`status`=2 then t.money end) as other_money 
,sum(case when t.capital_type != 5 AND  t.`status` = 2 then t.money end) as money
from db_digua_business.t_order_other t GROUP BY 1 ) too on too.order_id=om.id
-- WHERE tprm.order_create_time >= DATE_ADD(CURRENT_DATE, INTERVAL -12 DAY)
-- and tprm.order_create_time <= DATE_ADD(CURRENT_DATE, INTERVAL -3 DAY)
;
'''

df_zhys = query(sql)
df_zhys.shape

(59253, 65)

In [4]:
def qudao_type(a,b):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "社群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a

In [5]:
df_zhys["activity_source"] = df_zhys["activity_source"].fillna('未知活动') 
df_zhys["channel_name"] = df_zhys["channel_name"].fillna('未知渠道') 

df_zhys.loc[:,"归属渠道"]=df_zhys.apply(lambda x:qudao_type(x["channel_name"],x["activity_source"]),axis=1)

# 毛利计算系数

In [6]:
# def ygcz(row):
#     if 'iPhone' in row['product_name']:
#         if 'iPhone15' in row['product_name']:
#             return row['采购金额2'] * 0.4
#         else:
#             return row['采购金额2'] * 0.6
#     else:
#         return row['采购金额2'] * 0.425

# note 20241213修改
# 苹果16是0.6，其他苹果是0.4，安卓是0.425
def ygcz(row):
    if 'iPhone' in row['product_name']:
        if 'iPhone16' in row['product_name']:
            return row['采购金额2'] * 0.6
        else:
            return row['采购金额2'] * 0.4
    else:
        return row['采购金额2'] * 0.425

# 毛利计算

In [7]:
df_zhys.loc[:,"租赁方案"]=np.where(df_zhys["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")

In [9]:
# （1）rent_received：已收租金
# money：支付完成的非违约金的扣款金额（如滞纳金、赔偿金）
# has_actual：订单是否已买断
# paid_buyout_balance_payment：已归还残值/已付买断尾款
# liquidated_damages：违约金
# all_rental：总租金
# actual_gross_profit：实际毛利
# overdue_total_unreceived：逾期总未收
# （2）总已收2 = 因子1 + 因子2 + 因子3 + 因子4
# 因子1：已收租金
# 因子2：扣款金额（支付完成的非违约金的扣款金额）
# 因子3：订单状态非已完成，为已归还残值/已付买断尾款（订单未完成，paid_buyout_balance_payment 为0）；订单已完成，若未买断且已归还残值/已付买断尾款为0，则为预估残值2，否则为已归还残值/已付买断尾款
# 因子4：违约金
# （3）总应收2 = 因子1 + 因子2 + 因子3，违约金没有包含？
# 因子1：总租金
# 因子2：预估残值2
# 因子3：扣款金额（支付完成的非违约金的扣款金额）
# 租完即送的 总应收2 == 因子1 + 因子3
# （4）实际毛利2 = 因子1 - 因子2
# 因子2为采购成本
# ['已逾期', '已买断', '已完成'] 的：因子1 为 总已收2
# ['还款中', '续租中', '未到首期还款日'] 的：因子1 为 总应收2
# （5）应收毛利2 = 总应收2 - 采购成本
# （6）应收逾期率 = 逾期总未收/总应收2
# （7）成本逾期率 = 逾期损失/采购成本
# （8）应收毛利率 = 应收毛利2/采购成本
# （9）实际毛利率 = 实际毛利/采购成本
# （10）预计坏账率 = 逾期总未收/采购成本
# （11）实际毛利率 = 应收毛利率-预计坏账率

In [8]:
def profit_iphone(df_zhys):
        # note 20241213 修改
        # df_zhys = df_zhys[(df_zhys['purchase_amount']>0)&(df_zhys['订单状态']!='待收货')&(df_zhys['订单状态']!='已退款')&(df_zhys['租赁方案']!='租完即送')]
        df_zhys = df_zhys[(df_zhys['purchase_amount']>0)&(df_zhys['订单状态']!='待收货')&(df_zhys['订单状态']!='已退款')]
        df_zhys['product_name'] = df_zhys['model_number']+df_zhys['specification']
        df_zhys['下单日期'] = df_zhys['order_create_time'].dt.date
        df_zhys['下单月份'] = df_zhys['order_create_time'].dt.strftime('%Y-%m')
        df_zhys['下单年份'] = df_zhys['order_create_time'].dt.year
        df_zhys["内存"]=df_zhys["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
        df_zhys["颜色"]=df_zhys["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
        df_zhys['型号颜色1'] = df_zhys['model_number'].str.cat([df_zhys['内存'], df_zhys['颜色']], sep=' ').str.replace(' ', '')

        df_zhys['型号颜色1'] = df_zhys['型号颜色1'].dropna()
        # df_zhys['采购金额2'] = df_zhys['all_deposit']
        # all_deposit：总押金
        df_zhys['采购金额2'] = df_zhys['purchase_amount']
        # 数据过滤
        df_zhys = df_zhys[df_zhys['采购金额2']!=0]
        df_zhys['预估残值2'] = df_zhys.apply(ygcz, axis=1)
        # 总已收2
        # note 20241215 修改口径
        # df_zhys['总已收2'] = df_zhys['rent_received'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0) + np.where(df_zhys['订单状态']=='已完成',  
        #                                                         np.where((df_zhys['has_actual']==0), 
        #                                                                 np.where(df_zhys['paid_buyout_balance_payment']==0, df_zhys['预估残值2'], df_zhys['paid_buyout_balance_payment']), 
        #                                                         df_zhys['paid_buyout_balance_payment']), 
        #                                                 0) + np.where(df_zhys['liquidated_damages'].notna, df_zhys['liquidated_damages'], 0)

        # df_zhys['总已收2'] = np.where(df_zhys['租赁方案'] == '租完即送',df_zhys['rent_received'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0) +np.where(df_zhys['liquidated_damages'].notna, df_zhys['liquidated_damages'], 0),
        # df_zhys['rent_received'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0) + np.where(
        # df_zhys['订单状态'] == '已完成', np.where((df_zhys['has_actual'] == 0),
        #                                     np.where(df_zhys['paid_buyout_balance_payment'] == 0,df_zhys['预估残值2'],
        #                                 df_zhys['paid_buyout_balance_payment']), df_zhys['paid_buyout_balance_payment']),
        # 0) + np.where(df_zhys['liquidated_damages'].notna, df_zhys['liquidated_damages'], 0))
        
        df_zhys['总已收2'] = np.where(df_zhys['租赁方案'] == '租完即送',df_zhys['rent_received'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0) + np.where(df_zhys['订单状态'] == '已完成', df_zhys['paid_buyout_balance_payment'], 0) + np.where(df_zhys['liquidated_damages'].notna, df_zhys['liquidated_damages'], 0),
        df_zhys['rent_received'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0) + np.where(
        df_zhys['订单状态'] == '已完成', np.where((df_zhys['has_actual'] == 0),
                                            np.where(df_zhys['paid_buyout_balance_payment'] == 0,df_zhys['预估残值2'],
                                        df_zhys['paid_buyout_balance_payment']), df_zhys['paid_buyout_balance_payment']),
        0) + np.where(df_zhys['liquidated_damages'].notna, df_zhys['liquidated_damages'], 0))
        
        # 总应收2
        # note 20241215 修改口径
        # df_zhys['total_receivable2'] = df_zhys['all_rental'] + df_zhys['预估残值2'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0)
        df_zhys['total_receivable2'] = np.where(df_zhys.租赁方案 == '租完即送',(df_zhys['all_rental'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0)),(df_zhys['all_rental'] + df_zhys['预估残值2'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0)))


        status1_list = ['已逾期', '已买断', '已完成']
        status2_list = ['还款中', '续租中', '未到首期还款日']
        # 实际毛利2
        df_zhys['实际毛利2'] = np.where(df_zhys['rembursement_status'].isin(status2_list), df_zhys['total_receivable2']-df_zhys['purchase_amount'], 
                                np.where(df_zhys['rembursement_status'].isin(status1_list), df_zhys['总已收2']-df_zhys['purchase_amount'], df_zhys['actual_gross_profit']))
        # 应收毛利2
        df_zhys['gross_profit_receivable2'] = df_zhys['total_receivable2'] - df_zhys['purchase_amount']

        mon_zhys = df_zhys.groupby(['下单月份']).agg({'order_number':'size','purchase_amount':'sum','all_deposit':'sum','all_rental':'sum','buyout_balance_payment':'sum','total_receivable':'sum'
                                                ,'total_receivable2':'sum'
                                                ,'rent_received':'sum','unearned_rent_revenue':'sum','paid_buyout_balance_payment':'sum','total_received':'sum','total_unreceived':'sum'
                                                ,'unrecovered_cost':'sum','overdue_total_unreceived':'sum','overdue_loss':'sum','gross_profit_receivable': 'sum'
                                                ,'gross_profit_receivable2':'sum','actual_gross_profit':'sum'
                                                ,'采购金额2': 'sum','预估残值2':'sum', '总已收2': 'sum', '实际毛利2': 'sum'
                                        }).reset_index()
        mon_zhys['overdue_rate'] =   mon_zhys.overdue_total_unreceived/mon_zhys.total_receivable2    # 应收逾期率 = 逾期总未收/总应收2
        mon_zhys['loss_rate'] =   mon_zhys.overdue_loss/mon_zhys.purchase_amount                     # 成本逾期率 = 逾期损失/采购成本
        mon_zhys['receivable_profite_rate'] =   mon_zhys.gross_profit_receivable2/mon_zhys.purchase_amount    # 应收毛利率 = 应收毛利2/采购成本
        mon_zhys['profite_rate'] =   mon_zhys.实际毛利2/mon_zhys.purchase_amount      # 实际毛利率 = 实际毛利2/采购成本

        var = ['下单月份','order_number','purchase_amount', '采购金额2', 'all_deposit','all_rental','buyout_balance_payment','预估残值2', 'total_receivable2','total_receivable','rent_received'
               ,'unearned_rent_revenue','paid_buyout_balance_payment', '总已收2', 'total_received','total_unreceived','unrecovered_cost','overdue_total_unreceived','overdue_rate'
               ,'overdue_loss','loss_rate','gross_profit_receivable','gross_profit_receivable2','receivable_profite_rate','actual_gross_profit', '实际毛利2', 'profite_rate']
        mon_zhys2 = mon_zhys[var]

        mon_zhys2 = mon_zhys2.rename(columns={ 'search_time':'下单月份'
                                                ,'order_number':'订单数'
                                                ,'purchase_amount':'采购金额'
                                                ,'all_deposit':'总押金'
                                                ,'all_rental':'总租金'
                                                ,'buyout_balance_payment':'预估残值'
                                                ,'total_receivable':'总应收'
                                                ,'total_receivable2':'总应收2'
                                                ,'rent_received':'已收租金'
                                                ,'unearned_rent_revenue':'未收租金'
                                                ,'paid_buyout_balance_payment':'已归还残值/已付买断尾款'
                                                ,'total_received':'总已收'
                                                ,'total_unreceived':'总未收'
                                                ,'unrecovered_cost':'未回收成本'
                                                ,'overdue_total_unreceived':'逾期总未收'
                                                ,'overdue_rate':'应收逾期率'
                                                ,'overdue_loss':'逾期损失'
                                                ,'loss_rate':'成本逾期率'
                                                ,'gross_profit_receivable':'应收毛利'
                                                ,'gross_profit_receivable2':'应收毛利2'
                                                ,'receivable_profite_rate':'应收毛利率(应收毛利/采购成本)'
                                                ,'actual_gross_profit':'实际毛利'
                                                ,'profite_rate':'实际毛利率 (实际毛利/采购成本)'})
        mon_zhys2['预计坏账(逾期总未收/采购金额)'] = mon_zhys2['逾期总未收']/ mon_zhys2['采购金额']
        mon_zhys2['实际毛利(应收毛利-预计坏账)'] = mon_zhys2['应收毛利率(应收毛利/采购成本)'] - mon_zhys2['预计坏账(逾期总未收/采购金额)']
        # df_zhys['订单状态'].drop_duplicates()
        mon_zhys2['预估残值2'] = mon_zhys2['预估残值2'].apply(lambda x: format(x, '.2f'))
        mon_zhys2['应收毛利2'] = mon_zhys2['应收毛利2'].apply(lambda x: format(x, '.2f'))
        mon_zhys2['实际毛利2'] = mon_zhys2['实际毛利2'].apply(lambda x: format(x, '.2f'))

        # print (df_zhys[(df_zhys.order_number.isin(['A202207200249265','A202207181530165','A202207050851373','A2023050815084925','A202402130911067','A202312080320169','A202305192003535']))][['预估残值2','总已收2','total_receivable2','实际毛利2','gross_profit_receivable2']])
        return mon_zhys2

In [9]:
df_profit_all = profit_iphone(df_zhys)

In [12]:
df_profit_all[['下单月份', '采购金额', '采购金额2', '预估残值', '预估残值2','总应收', '总应收2','总已收', '总已收2', '应收毛利','应收毛利2','实际毛利', '实际毛利2']]#.to_excel('F:/月份毛利2.xlsx')

In [10]:
df_profit_all[['下单月份', '采购金额', '采购金额2', '预估残值', '预估残值2','总应收', '总应收2','总已收', '总已收2', '应收毛利','应收毛利2','实际毛利', '实际毛利2']]

,下单月份,采购金额,采购金额2,预估残值,预估残值2,总应收,总应收2,总已收,总已收2,应收毛利,应收毛利2,实际毛利,实际毛利2
0,2022-07,125405.00,125405.00,56523.43,50789.88,182555.48,1.606332e+05,125278.94,125278.95,57150.48,35228.24,2633.74,2326.15
1,2022-08,376541.00,376541.00,142725.03,151395.80,482686.18,4.610790e+05,340120.98,340120.98,106145.18,84537.96,-33933.42,-34215.02
2,2022-09,277982.75,277982.75,116206.97,111996.82,351061.75,3.431876e+05,298675.60,298675.60,73079.00,65204.85,23472.45,23136.85
3,2022-10,249543.09,249543.09,103498.64,100534.36,297219.33,2.942551e+05,265548.50,265548.50,47676.24,44712.00,20449.38,20415.81
4,2022-11,354218.81,354218.81,128408.58,142023.52,395032.28,3.964232e+05,370906.35,370906.35,40813.47,42204.40,16687.54,16687.54
5,2022-12,566276.59,566276.59,199746.17,227052.81,660888.72,6.596167e+05,590627.03,590627.03,94612.13,93340.14,24350.44,24350.44
6,2023-01,1533459.00,1533459.00,515244.37,616910.30,1912421.71,1.867556e+06,1674694.53,1674694.53,378962.71,334097.05,155615.73,155388.13
7,2023-02,2181552.82,2181552.82,906519.16,876898.03,2724072.32,2.616829e+06,2487438.00,2487438.00,542519.50,435276.15,326323.22,325444.02
8,2023-03,2083021.00,2083021.00,795907.44,839195.53,2534569.25,2.454493e+06,2294801.49,2294801.49,451548.25,371472.42,214379.88,214379.88
9,2023-04,4317651.50,4317651.50,1697754.67,1735236.25,5232019.21,5.021655e+06,4854319.89,4854319.89,914367.71,704003.84,581967.77,581460.17


In [14]:
# # 数据检查
# # df_zhys.订单状态.value_counts()
# # df_zhys.租赁方案.value_counts()
# df_zhys[(df_zhys['purchase_amount']>0)&(df_zhys['订单状态']!='待收货')&(df_zhys['订单状态']!='已退款')].租赁方案.value_counts()
# df_zhys[(df_zhys['purchase_amount']>0)&(df_zhys['订单状态']!='待收货')&(df_zhys['订单状态']!='已退款')].订单状态.value_counts()
# # df_zhys[df_zhys.purchase_amount > 0].all_deposit.max()
# # df_zhys[df_zhys.purchase_amount > 0].all_deposit.min()

# df_zhys['product_name'] = df_zhys['model_number']+df_zhys['specification']
# df_zhys['采购金额2'] = df_zhys['purchase_amount']
# df_zhys['预估残值2'] = df_zhys.apply(ygcz, axis=1)
# # df_zhys[df_zhys.product_name.str.contains('iPhone16')][['预估残值2','product_name','purchase_amount']]

# # has_actual ：订单是否已买断
# # purchase_amount ：采购成本
# # paid_buyout_balance_payment ：已归还残值/已付买断尾款
# # rent_received ：已收租金
# # all_rental ：总租金   --非续租的总租金
# # money ：支付完成的非违约金的扣款金额（如滞纳金、赔偿金）
# # liquidated_damages ：违约金
# # actual_gross_profit ：实际毛利
# # overdue_total_unreceived ：逾期总未收

# # 总租金 ！= 已收租金 + 逾期未归还
# # 实际毛利 ！= 已收租金 - 采购成本，可能还有溢价费、碎屏险等
# # 已完成的订单，已收租金不一定等于总租金--提前买断
# # 订单未完成，paid_buyout_balance_payment 为0


# df_zhys[(df_zhys['purchase_amount']>0)&(df_zhys['订单状态']!='待收货')&(df_zhys['订单状态']!='已退款')&(df_zhys['租赁方案']!='租完即送')].shape
# df_zhys[(df_zhys['purchase_amount']>0)&(df_zhys['订单状态']!='待收货')&(df_zhys['订单状态']!='已退款')].shape
# # search_time
# df_zhys['下单日期'] = df_zhys['order_create_time'].dt.date

# var1 = ['order_number','search_time','订单状态','租赁方案']
# var2 = ['has_actual','purchase_amount','paid_buyout_balance_payment','rent_received','all_rental','money','liquidated_damages','actual_gross_profit','overdue_total_unreceived']
# var_3 = ['预估残值2','总已收2','total_receivable2','实际毛利2','gross_profit_receivable2']
# con_1 = (df_zhys['purchase_amount']>0)&(df_zhys['订单状态']!='待收货')&(df_zhys['订单状态']!='已退款')
# con_2 = (df_zhys['租赁方案'] == '租完即送')
# # df_zhys[con_1 & con_2][var1 + var2].tail()

# # df_zhys['tmp'] = df_zhys.rent_received	+ df_zhys.overdue_total_unreceived
# # df_zhys[df_zhys.tmp != df_zhys.all_rental].shape
# # df_zhys[con_1 & con_2][var1 + var2][:2]
# # A202207200249265
# # A202207181530165	
# # A202207050851373
# # A2023050815084925 --续租，逾期中，逾期未回收<0 ？
# # A202402130911067  --逾期未回收>0，实际毛利>0
# # A202312080320169 --租完即送，已归还残值/已付买断尾款 >0，提取买断
# # A202305192003535  --已完成未买断

# # df_zhys[df_zhys.订单状态 != '已完成'].paid_buyout_balance_payment.min()
# # df_zhys[df_zhys.订单状态 != '已完成'].paid_buyout_balance_payment.max()
# # df_zhys[df_zhys.订单状态 == '已完成'].paid_buyout_balance_payment.min()


# df_zhys['总已收2'] = df_zhys['rent_received'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0) + np.where(df_zhys['订单状态']=='已完成',  
#                                                                 np.where((df_zhys['has_actual']==0), 
#                                                                         np.where(df_zhys['paid_buyout_balance_payment']==0, df_zhys['预估残值2'], df_zhys['paid_buyout_balance_payment']), 
#                                                                 df_zhys['paid_buyout_balance_payment']), 
#                                                         0) + np.where(df_zhys['liquidated_damages'].notna, df_zhys['liquidated_damages'], 0)
# # 总应收2
# df_zhys['total_receivable2'] = np.where(df_zhys.租赁方案 == '租完即送',(df_zhys['all_rental'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0)),(df_zhys['all_rental'] + df_zhys['预估残值2'] + np.where(df_zhys['money'].notna(), df_zhys['money'], 0)))

# status1_list = ['已逾期', '已买断', '已完成']
# status2_list = ['还款中', '续租中', '未到首期还款日']

# # df_zhys[con_1].rembursement_status.value_counts()
# # 已逾期         5928
# # 已买断         2570
# # 已完成          729

# # 还款中        13165
# # 未到首期还款日     2474
# # 续租中         2328

# # 检测中           70

# df_zhys['实际毛利2'] = np.where(df_zhys['rembursement_status'].isin(status2_list), df_zhys['total_receivable2']-df_zhys['purchase_amount'], 
#                                 np.where(df_zhys['rembursement_status'].isin(status1_list), df_zhys['总已收2']-df_zhys['purchase_amount'], df_zhys['actual_gross_profit']))
# # 应收毛利2
# df_zhys['gross_profit_receivable2'] = df_zhys['total_receivable2'] - df_zhys['purchase_amount']


# df_zhys[con_1& (df_zhys.order_number.isin(['A202207200249265','A202207181530165','A202207050851373','A2023050815084925','A202402130911067','A202312080320169','A202305192003535']))][var1 + var2 + var_3]